In [10]:
import pandas as pd
import librosa 
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from FeatureExtractor import FeatureExtractor
from DataFrameFactory import DataFrameFactory
from AudioDataLoader import AudioDataLoader

In [11]:
%matplotlib inline

In [33]:
FRAME_SIZE = 512
HOP_LENGTH = 256

In [12]:
sound_db = pd.read_csv('./db/UrbanSound8K.csv')

In [13]:
sound_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   slice_file_name  8732 non-null   object 
 1   fsID             8732 non-null   int64  
 2   start            8732 non-null   float64
 3   end              8732 non-null   float64
 4   salience         8732 non-null   int64  
 5   fold             8732 non-null   int64  
 6   classID          8732 non-null   int64  
 7   class            8732 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 545.9+ KB


In [14]:
sound_db.describe()

,fsID,start,end,salience,fold,classID
count,8732.000000,8732.000000,8732.000000,8732.000000,8732.000000,8732.000000
mean,116033.493816,38.645409,42.253312,1.347000,5.385937,4.592877
std,57991.017218,74.292126,74.369669,0.476043,2.846820,2.894544
min,344.000000,0.000000,0.105962,1.000000,1.000000,0.000000
25%,69942.250000,3.000000,6.839398,1.000000,3.000000,2.000000
50%,118279.000000,10.376492,14.000000,1.000000,5.000000,4.000000
75%,166942.000000,35.131372,38.866979,2.000000,8.000000,7.000000
max,209992.000000,600.125356,604.125356,2.000000,10.000000,9.000000


In [15]:
sound_db.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [16]:
class FeatureExtractor:
    @staticmethod
    def amplitude_envelope(signal: np.ndarray, frame_size: int, hop_length: int) -> np.array:
        ae = []
        for i in range(0, len(signal), hop_length):
            ae.append(max(signal[i:i + frame_size]))
        return np.array(ae)

    @staticmethod
    def root_mean_square(signal: np.ndarray, frame_size: int, hop_size: int) -> np.array:
        return librosa.feature.rms(y=signal, frame_length=frame_size, hop_length=hop_size)

    @staticmethod
    def normalize_feature(feature: np.ndarray) -> float:
        return np.sqrt(np.sum(feature ** 2))

In [17]:
class DataFrameFactory:
    def __init__(self) -> None:
        self.data_frame = pd.DataFrame()

    def add_column(self, name: str):
        self.data_frame[name] = None
        return self

    def build(self) -> pd.DataFrame:
        return self.data_frame

In [50]:
class AudioDataLoader:
    def load_from_df(self, df: pd.DataFrame):
        entries = []
        for i in range(len(df)):
            data = dict()
            file_name, fold, class_ = df.iloc[i][["slice_file_name", "fold", "class"]]
            
            if i < 10:
                y, sr = librosa.load(f'./db/fold{fold}/{file_name}')
                data['class'] = class_
                data['fold'] = fold
                data['amplitude_envelope'] = FeatureExtractor.normalize_feature(FeatureExtractor.amplitude_envelope(y, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH))
                
                entries.append(data)
        
        return entries

In [52]:
data_loader = AudioDataLoader()
new_data_list = data_loader.load_from_df(sound_db)
new_data_list

[{'class': 'dog_bark', 'fold': 5, 'amplitude_envelope': 2.5011067},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.20763576},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.14198396},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.2545674},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.15470709},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.17995471},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.12388224},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.13788542},
 {'class': 'children_playing', 'fold': 5, 'amplitude_envelope': 0.14960274},
 {'class': 'car_horn', 'fold': 10, 'amplitude_envelope': 0.889928}]